# Morphosyntax filter

In this notebook, we make a script that generate a json that contains the mask based on (1) morphosyntax and (2) stop-words filtering.
Two masks will be generated for each dataset.

## Folder setup 

The filter should be placed under the same cached dataset.
**TODO**
```
<root>
├── ProjectA
│   ├── A_map.json
│   ├── B_map.json
│   ├── C_map.json
│   └── ...
├── ProjectB
└── ...
```

## Setting up

In [1]:
%load_ext autoreload
%autoreload 2

from IPython.display import display, HTML
import sys
import os
from os import path

sys.path.append("./../src")

In [2]:
from modules.logger import init_logging
from modules.logger import log

init_logging(color=True)

# Check GPU
!nvidia-smi

Wed Oct  4 17:47:22 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.91.03    Driver Version: 460.91.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  On   | 00000000:04:00.0 Off |                  N/A |
| 23%   23C    P8     8W / 250W |    825MiB / 11178MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
df_legacy = pd.read_parquet(ROOT+"/dataset/esnli/test.pretransformed.parquet")
df_legacy.columns

In [16]:
df_legacy

,id,premise,hypothesis,label,explanation,highlight_premise,highlight_hypothesis,tokens.norm.premise,tokens.norm.hypothesis,rationale.premise,rationale.hypothesis,premise_heuristic,hypothesis_heuristic,tokens.form.premise,tokens.form.hypothesis,has_empty
0,2677109430.jpg#1r1n,This church choir sings to the masses as they ...,The church has cracks in the ceiling.,neutral,Not all churches have cracks in the ceiling,This church choir sings to the masses as they ...,The church has *cracks* *in* *the* *ceiling.*,"[this, church, choir, sing, to, the, masse, as...","[the, church, have, crack, in, the, ceiling, .]","[False, False, False, False, False, False, Fal...","[False, False, False, True, True, True, True, ...","[-1.0000000150474662e+30, 3.064525842666626, 1...","[-1.0000000150474662e+30, 7.628961086273193, -...","[This, church, choir, sings, to, the, masses, ...","[The, church, has, cracks, in, the, ceiling, .]",False
1,2677109430.jpg#1r1e,This church choir sings to the masses as they ...,The church is filled with song.,entailment,"""Filled with song"" is a rephrasing of the ""cho...",This church *choir* *sings* *to* *the* *masses...,The church is *filled* *with* *song.*,"[this, church, choir, sing, to, the, masse, as...","[the, church, be, fill, with, song, .]","[False, False, True, True, True, True, True, F...","[False, False, False, True, True, True, False]","[-1.0000000150474662e+30, 2.79181170463562, 2....","[-1.0000000150474662e+30, 7.628961086273193, -...","[This, church, choir, sings, to, the, masses, ...","[The, church, is, filled, with, song, .]",False
2,2677109430.jpg#1r1c,This church choir sings to the masses as they ...,A choir singing at a baseball game.,contradiction,A choir sing some other songs other than book ...,This church choir sings to the *masses* as the...,A choir *singing* at a *baseball* *game.*,"[this, church, choir, sing, to, the, masse, as...","[a, choir, singing, at, a, baseball, game, .]","[False, False, False, False, False, False, Tru...","[False, False, True, False, False, True, True,...","[-1.0000000150474662e+30, 2.5598974227905273, ...","[-1.0000000150474662e+30, 6.388305187225342, 6...","[This, church, choir, sings, to, the, masses, ...","[A, choir, singing, at, a, baseball, game, .]",False
3,6160193920.jpg#4r1n,"A woman with a green headscarf, blue shirt and...",The woman is young.,neutral,the woman could've been old rather than young,"A woman with a green headscarf, blue shirt and...",The woman is *young.*,"[a, woman, with, a, green, headscarf, ,, blue,...","[the, woman, be, young, .]","[False, False, False, False, False, False, Fal...","[False, False, False, True, False]","[-1.0000000150474662e+30, 2.597653388977051, -...","[-1.0000000150474662e+30, 5.648240089416504, -...","[A, woman, with, a, green, headscarf, ,, blue,...","[The, woman, is, young, .]",False
4,6160193920.jpg#4r1e,"A woman with a green headscarf, blue shirt and...",The woman is very happy.,entailment,a grin suggests hapiness.,"A woman with a green headscarf, blue shirt and...",The woman is very *happy.*,"[a, woman, with, a, green, headscarf, ,, blue,...","[the, woman, be, very, happy, .]","[False, False, False, False, False, False, Fal...","[False, False, False, False, True, False]","[-1.0000000150474662e+30, 2.784580707550049, -...","[-1.0000000150474662e+30, 5.648240089416504, -...","[A, woman, with, a, green, headscarf, ,, blue,...","[The, woman, is, very, happy, .]",False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9819,4378810163.jpg#4r1c,Two women are observing something together.,Two women are standing with their eyes closed.,contradiction,Answer: The women cannot be observing with eye...,Two women are *observing* something together.,Two women are standing with their *eyes* *clos...,"[two, woman, be, observe, something, together, .]","[two, woman, be, stand, with, their, eye, clos...","[False, False, False, True, False, False, False]","[False, False, False, False, False, False, Tru...","[-1.0

In [3]:
import pandas as pd
import spacy
import platform

# Define the path based on the current node
node = platform.node()
log.info(f'Current node: {node}')
if node == 'MAC-C02D80HRMD6':
    ROOT = '/Users/dunguyen/Developer/server_backup/historic/2023-06-05'
else:
    ROOT = '/home/dunguyen/RUNS'

DATASET = 'hatexplain'

# Load cached datasets
# TODO make the path adapt for each dataset
df = pd.read_parquet(ROOT+"/dataset_/hatexplain/test.parquet")
# TODO sample for dev
df = df.sample(1000)

# Get preprocessed tokens from the dataset
token_forms = df['tokens.form'].tolist()

# Contruction of tagger pipeline
sm = spacy.load("en_core_web_sm")
#tagger = sm.add_pipe("tagger")

# Convert each sentence into a spacy doc
from spacy.tokens import Doc
docs = [Doc(sm.vocab, words=tokens) for tokens in token_forms]

04-10-2023 17:47:23 |     INFO   2659519722.py:<cell line: 7>:7  Current node: grele-3.nancy.grid5000.fr


In [4]:
# Reduce columns:
df = df[['id', 'tokens.form', 'tokens.norm', 'label']]

In [5]:
# Process the docs
postags = []
stopwords = []

for doc in sm.pipe(docs, batch_size=50):
    pos_ = [token.pos_ for token in doc]
    is_stop = [token.is_stop for token in doc]
    postags.append(pos_)    
    stopwords.append(is_stop)
    
df['tokens.pos'] = postags
df['tokens.stops'] = stopwords

def morpho_mask(row):
    pos = row['tokens.pos']
    nva_ = [p in ['NOUN', 'VERB', 'ADJ'] for p in pos]
    stops_ = row['tokens.stops']
    mask = [(p and not s) for p, s in zip(nva_, stops_)]
    row['morpho_mask'] = mask
    return row

df = df.apply(morpho_mask, axis=1)
df.to_parquet(ROOT+"/dataset_/hatexplain/test.morpho_mask.parquet")

log.info(f'Saved to {ROOT+"/dataset_/hatexplain/test.morpho_mask.parquet"}')